# Import the libraries

In [1]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
import shap  # package used to calculate Shap values
import time
import glob
import sys
import os

# Python pre-requisites

In [2]:
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

# Creating data pipeline

In [10]:
path = 'D:\ISU\IE 587\Project'

## Loading raw data

### Train data

In [5]:
%%time
train = pd.read_csv('train.csv' , dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

Wall time: 1min 15s


### Data shape

In [6]:
train.shape

(629145480, 2)

### Submission file

In [7]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
submission.head()

,time_to_failure
seg_id,
seg_00030f,0
seg_0012b5,0
seg_00184e,0
seg_003339,0
seg_0042cc,0


In [9]:
submission.shape

(2624, 1)

### Test data

In [11]:
len(os.listdir(os.path.join(path, 'test')))

2624

We have 2624 files in test dataset and 2624 rows in submission file. Thus, it is clear that we have to make one submission for each test file.

# Data preprocessing

## Creating segmentation

In [12]:
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))
segments

4194

## Creating an empty train dataset with rows equal to segments

In [13]:
X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min','sum','skew','kurt'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

## Creating new features

In [14]:
for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()
    X_train.loc[segment, 'sum'] = x.sum()
    X_train.loc[segment, 'skew'] =skew(x)
    X_train.loc[segment, 'kurt'] = kurtosis(x)

100%|█████████████████████████████████████████████████████████████████████████████| 4194/4194 [00:38<00:00, 107.89it/s]


Awesome! We have now filled the empty datasets we created with aggregations (mean, standard deviation, etc) of segments. 

## Snapshot of created features

In [16]:
X_train.head()

,ave,std,max,min,sum,skew,kurt
0,4.884113333333334,5.101089126891323,104.0,-98.0,732617.0,-0.024060926015874,33.661319221480703
1,4.725766666666667,6.588801819164257,181.0,-154.0,708865.0,0.390556598755942,98.755185259155695
2,4.906393333333333,6.967373808828945,140.0,-106.0,735959.0,0.217388387534031,33.554052910588105
3,4.902240000000000,6.922282112791032,197.0,-199.0,735336.0,0.757269963614698,116.544246785098039
4,4.908720000000000,7.301085852684289,145.0,-126.0,736308.0,0.064530423958889,52.976098920993913


In [17]:
y_train.head()

,time_to_failure
0,1.430797219276428
1,1.391498923301697
2,1.353196144104004
3,1.313797831535339
4,1.274399518966675


## Check if there any missing values

In [20]:
X_train.isna().sum()

ave     0
std     0
max     0
min     0
sum     0
skew    0
kurt    0
dtype: int64

In [21]:
y_train.isna().sum()

time_to_failure    0
dtype: int64

## Now we must create our X_test for making predictions

In [24]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)
X_test.head()

,ave,std,max,min,sum,skew,kurt
seg_id,,,,,,,
seg_00030f,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_0012b5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_00184e,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_003339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_0042cc,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating test features

In [29]:
%%time
for seg_id in  tqdm(X_test.index):
    seg = pd.read_csv('D:\\ISU\\IE 587\\Project\\test\\' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[seg_id, 'sum'] = x.sum()
    X_test.loc[seg_id, 'skew'] =skew(x)
    X_test.loc[seg_id, 'kurt'] = kurtosis(x)

100%|██████████████████████████████████████████████████████████████████████████████| 2624/2624 [02:30<00:00, 17.40it/s]


Wall time: 2min 30s


## Feature scaling

We have all of the data frames for applying ML algorithms. Now we are just adding some feature scaling.

In [30]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [31]:
X_test_scaled = scaler.transform(X_test)

In [32]:
X=X_train.copy()
y=y_train.copy()

Now that we are done with data preprocessing, let's head straight into the predictive analysis.

# Predictive Analysis

## SVM (Support Vector Machine)

In [33]:
svm = NuSVR()
svm.fit(X_train_scaled, y_train.values.flatten())
y_pred_svm = svm.predict(X_train_scaled)

In [34]:
score = mean_absolute_error(y_train.values.flatten(), y_pred_svm)
print(f'Score: {score:0.3f}')

Score: 2.286


## LGBM (Light Gradient Boosting Machine)

In [35]:
folds = KFold(n_splits=5, shuffle=True, random_state=42)

In [36]:
params = {'objective' : "regression", 
               'boosting':"gbdt",
               'metric':"mae",
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.001,
               'num_leaves' : 52,
               'max_depth':-1,
               'tree_learner' : "serial",
               'feature_fraction' : 0.85,
               'bagging_freq' : 1,
               'bagging_fraction' : 0.85,
               'min_data_in_leaf' : 10,
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : -1}

In [37]:
%%time
y_pred_lgb = np.zeros(len(X_test_scaled))
for fold_n, (train_index, valid_index) in tqdm(enumerate(folds.split(X))):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
    model = lgb.LGBMRegressor(**params, n_estimators = 22000, n_jobs = -1)
    model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='mae',
                    verbose=1000, early_stopping_rounds=200)
            
    y_pred_valid = model.predict(X_valid)
    y_pred_lgb += model.predict(X_test_scaled, num_iteration=model.best_iteration_) / folds.n_splits

0it [00:00, ?it/s]

Fold 0 started at Wed Oct 30 18:33:51 2019
Training until validation scores don't improve for 200 rounds
[1000]	training's l1: 2.63947	valid_1's l1: 2.72916
[2000]	training's l1: 1.91945	valid_1's l1: 2.2434
Early stopping, best iteration is:
[2595]	training's l1: 1.80144	valid_1's l1: 2.22492


1it [00:04,  4.64s/it]

Fold 1 started at Wed Oct 30 18:33:56 2019
Training until validation scores don't improve for 200 rounds
[1000]	training's l1: 2.63386	valid_1's l1: 2.74593
[2000]	training's l1: 1.91446	valid_1's l1: 2.23817
Early stopping, best iteration is:
[2776]	training's l1: 1.7721	valid_1's l1: 2.20341


2it [00:09,  4.74s/it]

...

Fold 3 started at Wed Oct 30 18:34:05 2019
Training until validation scores don't improve for 200 rounds
[1000]	training's l1: 2.62758	valid_1's l1: 2.76779
[2000]	training's l1: 1.92424	valid_1's l1: 2.16267
[3000]	training's l1: 1.75167	valid_1's l1: 2.11944
Early stopping, best iteration is:
[3039]	training's l1: 1.74704	valid_1's l1: 2.11922


4it [00:19,  4.87s/it]

Fold 4 started at Wed Oct 30 18:34:11 2019
Training until validation scores don't improve for 200 rounds
[1000]	training's l1: 2.61837	valid_1's l1: 2.81991
[2000]	training's l1: 1.90964	valid_1's l1: 2.27678
[3000]	training's l1: 1.74039	valid_1's l1: 2.23655
Early stopping, best iteration is:
[3284]	training's l1: 1.70879	valid_1's l1: 2.23575


5it [00:25,  5.06s/it]


Wall time: 25.5 s


## Catboost

In [38]:
train_pool = Pool(X,y)
cat_model = CatBoostRegressor(
                               iterations=3000,# change 25 to 3000 to get best performance 
                               learning_rate=0.03,
                               eval_metric='MAE',
                              )
cat_model.fit(X,y,silent=True)
y_pred_cat = cat_model.predict(X_test)

In [40]:
print(cat_model.get_best_score())

{'learn': {'MAE': 1.3321847516511751, 'RMSE': 1.7080476247192358}}


# Submission for LGBM

In [41]:
submission['time_to_failure'] = y_pred_lgb
submission.to_csv('submission_lgb.csv')